# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

In [1]:
# We have a dataset about cars with column such as : 
# id, region, price, year, manufacturer, model, condition, cylinders, fuel, odometer, title_status, transmission, 
# VIN, drive, size, type, paint_color, state

In [2]:
# we want to know based on those data attributes, which attributes influence the price and what are the relationship between those attributes and 
# the price of the car
# we will import libraries and load the dataset here:

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn import linear_model
import plotly.graph_objects as go
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import shuffle
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [4]:
vehicle_data = pd.read_csv('data/vehicles.csv')
vehicle_data.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [5]:
# We determine how many missing values are in each column
# The total number of rows : 426880
serie = vehicle_data.isnull().sum()
print(serie)

id                   0
region               0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
state                0
dtype: int64


In [6]:
# we see that there are many missing values so I could consider trying to fill empty values with default values
# in the case of empty manufacturer values, I could consider trying to derive the manufacturer from the model value when it is possible

In [7]:
vehicle_data['price'].describe()

count    4.268800e+05
mean     7.519903e+04
std      1.218228e+07
min      0.000000e+00
25%      5.900000e+03
50%      1.395000e+04
75%      2.648575e+04
max      3.736929e+09
Name: price, dtype: float64

In [8]:
# we see that the price range from 0 to 3.736929e+09 and the average price is  75199.03e+04
# the prices greater than 2000000 dont seem too realistic so I could consider removing the rows with price greater than 2000000

In [9]:
categorical_columns = vehicle_data.select_dtypes(include=['object', 'category']).columns
for col in categorical_columns:
    df_model_count = vehicle_data.groupby(col, as_index=False)['id'].count().rename(columns = {'id':'count'}).sort_values(by = 'count', ascending = False)
    plt = px.histogram(df_model_count, x = col, y = 'count', nbins = 10)
    plt.update_layout(title=f'Count by {col}', yaxis_title="Count", title_x = 0.5)
    #plt.show()
    # I disabled the image otherwise the size of the ipynb file seems to be too big and i can't upload it to github

In [10]:
for col in categorical_columns:
    if col not in ('model'):
        fig = px.histogram(vehicle_data, x=col, y="price", histfunc='avg')
        # Sort bars by avg in descending order
        fig.update_layout(xaxis={'categoryorder': 'total descending'}, title=f'Average of price by {col}', title_x = 0.5)
        #fig.show()
        # I disabled the image otherwise the size of the ipynb file seems to be too big and i can't upload it to github

In [11]:
# we observe that :
# the most common manufacturer is ford, chevrolet and toyota
# the most common model is the f-150
# conditions can be good, excellent, like new, faire, new and salvage
# number of cylinders can be 3, 4, 5, 6, 8, 10, 12 or other
# fuel can be gas, diesel, hybrid, electric or other
# title_status can be clean, rebuilt, salvage, lien, missing parts only
# transmission can be automatic, other, manual
# VIN have lot of possible values
# drive can be 4wd, fwd, rwd
# size can be full-size, mid-size, compact, sub-compact
# type can be sedan, SUV, pickup, truck, coupe, hatchback, wagon, van, convertible, mini-van, offroad, bus or other
# the most commom paint color is white
# the most common state is CA

In [12]:
# In the case of cylinders I could try to replace the string values by numerical values as it seems to make sense

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [13]:
# we clean the missing data

# column model: replace blank by Other
vehicle_data['model'] = vehicle_data['model'].fillna('Other')
# column condition: replace blank by missing
vehicle_data['condition'] = vehicle_data['condition'].fillna('missing')
# column cylinders: replace blank by other
vehicle_data['cylinders'] = vehicle_data['cylinders'].fillna('other')
# column fuel: replace blank by other
vehicle_data['fuel'] = vehicle_data['fuel'].fillna('other')
# column title_status: replace blank by missing
vehicle_data['title_status'] = vehicle_data['title_status'].fillna('missing')
# column transmission: replace blank by other
vehicle_data['transmission'] = vehicle_data['transmission'].fillna('other')
# column VIN: replace blank by 0
vehicle_data['VIN'] = vehicle_data['VIN'].fillna('0')
# column drive: replace blank by missing
vehicle_data['drive'] = vehicle_data['drive'].fillna('missing')
# column size: replace blank by missing
vehicle_data['size'] = vehicle_data['size'].fillna('missing')
# column VIN: replace blank by other
vehicle_data['type'] = vehicle_data['type'].fillna('other')
# column paint_color: replace blank by missing
vehicle_data['paint_color'] = vehicle_data['paint_color'].fillna('missing')

# Guess what should be the missing values in column manufacturer based on the value in column model

# Get unique values from column 'manufacturer'
manufacturer_unique_values = vehicle_data['manufacturer'].dropna().unique()
manufacturer_unique_values_list = manufacturer_unique_values.tolist()
upper_manufacturer_unique_values_list = [item.upper() for item in manufacturer_unique_values_list]
upper_manufacturer_unique_values_list.remove('RAM')

for manufacturer in upper_manufacturer_unique_values_list:
  vehicle_data['manufacturer'] = np.where(vehicle_data['manufacturer'].isnull() & vehicle_data['model'].str.upper().str.contains(manufacturer), manufacturer, vehicle_data['manufacturer'])

# remove rows where the price is more than 2000000
vehicle_data = vehicle_data[vehicle_data['price'] < 2000000]
#vehicle_data = vehicle_data[vehicle_data['price'] < 100000]

# In column cylinders, replace string values by numbers
vehicle_data["cylinders"] = vehicle_data["cylinders"].str.replace("3 cylinders","3")
vehicle_data["cylinders"] = vehicle_data["cylinders"].str.replace("4 cylinders","4")
vehicle_data["cylinders"] = vehicle_data["cylinders"].str.replace("5 cylinders","5")
vehicle_data["cylinders"] = vehicle_data["cylinders"].str.replace("6 cylinders","6")
vehicle_data["cylinders"] = vehicle_data["cylinders"].str.replace("8 cylinders","8")
vehicle_data["cylinders"] = vehicle_data["cylinders"].str.replace("10 cylinders","10")
vehicle_data["cylinders"] = vehicle_data["cylinders"].str.replace("12 cylinders","12")
vehicle_data["cylinders"] = vehicle_data["cylinders"].str.replace("other","0")


# add new column "decade" based on the year
vehicle_data['decade'] = np.floor(vehicle_data["year"] / 10) * 10

# we drop the rest of the data as we were not able to populate the missing values in some columns
vehicle_data = vehicle_data.dropna()

In [14]:
# Save final dataset to a CSV file
vehicle_data.to_csv('final_dataset.csv', index=False)  # Set index=False to exclude the index column

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [15]:
#px.scatter(data_frame=vehicle_data, x = 'year', y = 'price')

# i disabled this image as it makes my computer slow

In [16]:
# 1) Model with simple linear regression with column year with fit_intercept = False
features = vehicle_data[['year']]
price = vehicle_data['price']
f = linear_model.LinearRegression(fit_intercept = False)
f.fit(features, price)
vehicle_data['prediction'] = f.predict(vehicle_data[['year']])

#fig = go.Figure()
#fig.add_trace(go.Scatter(x=vehicle_data['year'], y = vehicle_data['price'], 
#                         mode = "markers", name = "actual"))
#fig.add_trace(go.Scatter(x=vehicle_data['year'], y = vehicle_data['prediction'], 
#                         mode = "lines", name = "predicted"))
#fig.update_layout(font_size = 20)

# i disabled this image as it makes my computer slow

In [17]:
f.coef_

array([8.72637321])

In [18]:
f.intercept_

0.0

In [19]:
# Computing the L2 Loss and MSE
# add loss to the dataframe
vehicle_data["L2_loss"] = (vehicle_data["prediction"] - vehicle_data["price"])**2
vehicle_data.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,...,transmission,VIN,drive,size,type,paint_color,state,decade,prediction,L2_loss
27,7316814884,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8,gas,57923.0,...,other,3GTP1VEC4EG551563,missing,missing,pickup,white,al,2010.0,17574.915638,2.564829e+08
28,7316814758,auburn,22590,2010.0,chevrolet,silverado 1500,good,8,gas,71229.0,...,other,1GCSCSE06AZ123805,missing,missing,pickup,blue,al,2010.0,17540.010145,2.550240e+07
29,7316814989,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8,gas,19160.0,...,other,3GCPWCED5LG130317,missing,missing,pickup,red,al,2020.0,17627.273877,4.823613e+08
30,7316743432,auburn,30990,2017.0,toyota,tundra double cab sr,good,8,gas,41124.0,...,other,5TFRM5F17HX120972,missing,missing,pickup,red,al,2010.0,17601.094757,1.792628e+08
31,7316356412,auburn,15000,2013.0,ford,f-150 xlt,excellent,6,gas,128000.0,...,automatic,0,rwd,full-size,truck,black,al,2010.0,17566.189264,6.585327e+06


In [20]:
#compute the mean
np.mean(vehicle_data["L2_loss"])

309197835.9520567

In [21]:
# 2) Model with simple linear regression with column year with fit_intercept = True
features = vehicle_data[['year']]
price = vehicle_data['price']
f = linear_model.LinearRegression(fit_intercept = True)
f.fit(features, price)
vehicle_data['prediction'] = f.predict(vehicle_data[['year']])

#fig = go.Figure()
#fig.add_trace(go.Scatter(x=vehicle_data['year'], y = vehicle_data['price'], 
#                         mode = "markers", name = "actual"))
#fig.add_trace(go.Scatter(x=vehicle_data['year'], y = vehicle_data['prediction'], 
#                         mode = "lines", name = "predicted"))
#fig.update_layout(font_size = 20)

# i disabled this image as it makes my computer slow

In [22]:
f.coef_

array([480.02089827])

In [23]:
f.intercept_

-948024.2862590244

In [24]:
vehicle_data["L2_loss"] = (vehicle_data["prediction"] - vehicle_data["price"])**2
vehicle_data.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,...,transmission,VIN,drive,size,type,paint_color,state,decade,prediction,L2_loss
27,7316814884,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8,gas,57923.0,...,other,3GTP1VEC4EG551563,missing,missing,pickup,white,al,2010.0,18737.802858,2.205878e+08
28,7316814758,auburn,22590,2010.0,chevrolet,silverado 1500,good,8,gas,71229.0,...,other,1GCSCSE06AZ123805,missing,missing,pickup,blue,al,2010.0,16817.719265,3.331922e+07
29,7316814989,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8,gas,19160.0,...,other,3GCPWCED5LG130317,missing,missing,pickup,red,al,2020.0,21617.928247,3.229954e+08
30,7316743432,auburn,30990,2017.0,toyota,tundra double cab sr,good,8,gas,41124.0,...,other,5TFRM5F17HX120972,missing,missing,pickup,red,al,2010.0,20177.865553,1.169023e+08
31,7316356412,auburn,15000,2013.0,ford,f-150 xlt,excellent,6,gas,128000.0,...,automatic,0,rwd,full-size,truck,black,al,2010.0,18257.781959,1.061314e+07


In [25]:
#compute the mean
np.mean(vehicle_data["L2_loss"])

291390766.6010254

In [26]:
# 3) Model with multiple Linear Regression

In [27]:
# fit a model on year and cylinders and fit_intercept = True
two_features = vehicle_data[["year", "cylinders"]]
price = vehicle_data["price"]

f2 = linear_model.LinearRegression(fit_intercept = True)
f2.fit(two_features, price)

LinearRegression()

In [28]:
#show coefficients
f2.coef_

array([494.07256164, 259.20964852])

In [29]:
# make a prediction for a car from 2024 and 10 cylinders
f2.predict([[2024, 10]])

C:\Users\vince\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



array([25400.88920021])

In [30]:
vehicle_data["prediction_2d"] = f2.predict(vehicle_data[["year", "cylinders"]])
vehicle_data.head(5)

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,...,VIN,drive,size,type,paint_color,state,decade,prediction,L2_loss,prediction_2d
27,7316814884,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8,gas,57923.0,...,3GTP1VEC4EG551563,missing,missing,pickup,white,al,2010.0,18737.802858,2.205878e+08,19941.744287
28,7316814758,auburn,22590,2010.0,chevrolet,silverado 1500,good,8,gas,71229.0,...,1GCSCSE06AZ123805,missing,missing,pickup,blue,al,2010.0,16817.719265,3.331922e+07,17965.454040
29,7316814989,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8,gas,19160.0,...,3GCPWCED5LG130317,missing,missing,pickup,red,al,2020.0,21617.928247,3.229954e+08,22906.179657
30,7316743432,auburn,30990,2017.0,toyota,tundra double cab sr,good,8,gas,41124.0,...,5TFRM5F17HX120972,missing,missing,pickup,red,al,2010.0,20177.865553,1.169023e+08,21423.961972
31,7316356412,auburn,15000,2013.0,ford,f-150 xlt,excellent,6,gas,128000.0,...,0,rwd,full-size,truck,black,al,2010.0,18257.781959,1.061314e+07,18929.252428


In [31]:
# px.scatter_3d(vehicle_data, x = "year", y = "cylinders", z = "price")
# i disabled this image as it makes my computer slow

In [32]:
mean_squared_error(vehicle_data["price"], f2.predict(vehicle_data[["year", "cylinders"]]))

290725820.39870703

In [33]:
# 4) Model with Linear regression using Non-numeric Features

In [34]:
# Let's create a copy of the dataset that only has 3 features in order to keep things simple
three_features = ['odometer', 'cylinders', 'manufacturer']

three_feature_data = pd.DataFrame(vehicle_data[three_features])
three_feature_data.head(5)

,odometer,cylinders,manufacturer
27,57923.0,8,gmc
28,71229.0,8,chevrolet
29,19160.0,8,chevrolet
30,41124.0,8,toyota
31,128000.0,6,ford


In [35]:
# let's create "dummies" that represent whether the manufacturer
dummies = pd.get_dummies(three_feature_data['manufacturer'])
data_w_dummies = pd.concat([three_feature_data, dummies], axis=1)
del data_w_dummies["manufacturer"]
data_w_dummies.iloc[[193, 90, 25, 26, 190], :]

,odometer,cylinders,ACURA,AUDI,BMW,BUICK,CADILLAC,CHEVROLET,CHRYSLER,DODGE,...,pontiac,porsche,ram,rover,saturn,subaru,tesla,toyota,volkswagen,volvo
228,239000.0,8,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
118,8490.0,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
52,2195.0,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
53,10688.0,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
225,137701.0,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
# we can fit our model.
f_with_manufacturer = linear_model.LinearRegression(fit_intercept=True)
f_with_manufacturer.fit(data_w_dummies, price)


LinearRegression()

In [37]:
f_with_manufacturer.coef_

array([-1.16870349e-02, -8.78968245e+01, -7.10440898e+03, -1.27212834e+04,
       -8.65244186e+03,  6.43940544e+04,  1.13063318e+04,  1.67595938e+03,
       -7.89916451e+03, -7.63890259e+03, -9.19356187e+03, -2.00238037e+03,
        6.31648215e+02, -1.03996891e+04, -1.06694353e+04, -4.53224278e+03,
       -8.84818417e+03, -5.29669804e+03, -1.24129570e+04, -9.93404195e+03,
       -9.08226346e+03,  1.38940057e+04,  6.49709516e+03, -1.16506339e+04,
       -1.10455028e+04, -1.83351046e+04, -9.43958864e+03, -1.27939260e+04,
       -1.03976356e+04, -1.74465142e+03,  2.27531533e+03,  9.77160959e+03,
        3.55374455e+04,  5.80055889e+03,  1.36451815e+03, -2.96378211e+03,
        1.93842537e+03,  1.52624443e+03, -6.79708853e+03, -2.48122982e+03,
       -2.87015873e+03,  8.95041113e+04, -6.15988290e+03,  2.08247393e+03,
        5.99791169e+03, -5.83494853e+03, -6.41383121e+03, -6.81789803e+03,
        2.02057434e+03,  8.57894033e+03,  7.28435227e+02, -6.53678255e+03,
       -9.82983194e+03,  

In [38]:
## Let's compute the MSE
mean_squared_error(vehicle_data['price'], f_with_manufacturer.predict(data_w_dummies))

282145130.9889208

In [39]:
# px.scatter(vehicle_data, x = "odometer", y =  "price", color = "manufacturer", trendline = "ols")
# I disabled the image otherwise the size of the ipynb file seems to be too big and i can't upload it to github

In [40]:
# We create the training data set and the dev data set
all_indices = range(0, len(vehicle_data))
all_indices = shuffle(all_indices)
training_indices, dev_indices = np.split(all_indices, [330000])
training_vehicle_data = vehicle_data.iloc[training_indices]
dev_vehicle_data = vehicle_data.iloc[dev_indices]

In [41]:
# 5) Model using a SKLearn Pipeline and polynomial features and only 1 attribute
pipelined_model_1 = Pipeline([
    ('vincent_transform_1', PolynomialFeatures(degree = 3, include_bias = False)),
    ('vincent_regression_1', linear_model.LinearRegression())
])
pipelined_model_1.fit(vehicle_data[["year"]], vehicle_data["price"])

Pipeline(steps=[('vincent_transform_1',
                 PolynomialFeatures(degree=3, include_bias=False)),
                ('vincent_regression_1', LinearRegression())])

In [42]:
pipelined_model_1.predict([[2021]])

C:\Users\vince\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but PolynomialFeatures was fitted with feature names



array([33942.02301121])

In [43]:
pipelined_model_1.named_steps['vincent_regression_1'].coef_

array([ 6.17925858e+06, -3.13304795e+03,  5.29432578e-01])

In [44]:
mean_squared_error(pipelined_model_1.predict(training_vehicle_data[["year"]]), training_vehicle_data["price"])

245444482.86681846

In [45]:
#fig = px.scatter(vehicle_data, x = "year", y = "price")
#fig.show()
# i disabled the image because it makes my computer slow

In [46]:
# 6) Model using a SKLearn Pipeline and polynomial features and many attributes
numeric_features = ["year", "cylinders", "odometer"]
pipelined_model_2 = Pipeline([
    ('vincent_transform_2', PolynomialFeatures(degree = 3, include_bias = False)),
    ('vincent_regression_2', linear_model.LinearRegression())
])
pipelined_model_2.fit(training_vehicle_data[numeric_features], training_vehicle_data["price"])

Pipeline(steps=[('vincent_transform_2',
                 PolynomialFeatures(degree=3, include_bias=False)),
                ('vincent_regression_2', LinearRegression())])

In [47]:
mean_squared_error(pipelined_model_2.predict(training_vehicle_data[numeric_features]), training_vehicle_data["price"])

247035445.96211284

In [48]:
# 7) Model using SequentialFeatureSelection
poly3 = PolynomialFeatures(degree = 3, include_bias = False)
all_degree_3_combinations = poly3.fit_transform(vehicle_data[["year", "cylinders", "odometer"]])
all_degree_3_combinations = pd.DataFrame(all_degree_3_combinations, columns = poly3.get_feature_names_out())

In [49]:
all_degree_3_combinations

,year,cylinders,odometer,year^2,year cylinders,year odometer,cylinders^2,cylinders odometer,odometer^2,year^3,year^2 cylinders,year^2 odometer,year cylinders^2,year cylinders odometer,year odometer^2,cylinders^3,cylinders^2 odometer,cylinders odometer^2,odometer^3
0,2014.0,8.0,57923.0,4056196.0,16112.0,116656922.0,64.0,463384.0,3.355074e+09,8.169179e+09,32449568.0,2.349470e+11,128896.0,9.332554e+08,6.757119e+12,512.0,3707072.0,2.684059e+10,1.943359e+14
1,2010.0,8.0,71229.0,4040100.0,16080.0,143170290.0,64.0,569832.0,5.073570e+09,8.120601e+09,32320800.0,2.877723e+11,128640.0,1.145362e+09,1.019788e+13,512.0,4558656.0,4.058856e+10,3.613853e+14
2,2020.0,8.0,19160.0,4080400.0,16160.0,38703200.0,64.0,153280.0,3.671056e+08,8.242408e+09,32643200.0,7.818046e+10,129280.0,3.096256e+08,7.415533e+11,512.0,1226240.0,2.936845e+09,7.033743e+12
3,2017.0,8.0,41124.0,4068289.0,16136.0,82947108.0,64.0,328992.0,1.691183e+09,8.205739e+09,32546312.0,1.673043e+11,129088.0,6.635769e+08,3.411117e+12,512.0,2631936.0,1.352947e+10,6.954823e+13
4,2013.0,6.0,128000.0,4052169.0,12078.0,257664000.0,36.0,768000.0,1.638400e+10,8.157016e+09,24313014.0,5.186776e+11,72468.0,1.545984e+09,3.298099e+13,216.0,4608000.0,9.830400e+10,2.097152e+15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405657,2019.0,6.0,32226.0,4076361.0,12114.0,65064294.0,36.0,193356.0,1.038515e+09,8.230173e+09,24458166.0,1.313648e+11,72684.0,3.903858e+08,2.096762e+12,216.0,1160136.0,6.231090e+09,3.346719e+13
405658,2020.0,0.0,12029.0,4080400.0,0.0,24298580.0,0.0,0.0,1.446968e+08,8.242408e+09,0.0,4.908313e+10,0.0,0.000000e+00,2.922876e+11,0.0,0.0,0.000000e+00,1.740558e+12
405659,2020.0,0.0,4174.0,4080400.0,0.0,8431480.0,0.0,0.0,1.742228e+07,8.242408e+09,0.0,1.703159e+10,0.0,0.000000e+00,3.519300e+10,0.0,0.0,0.000000e+00,7.272058e+10
405660,2018.0,6.0,30112.0,4072324.0,12108.0,60766016.0,36.0,180672.0,9.067325e+08,8.217950e+09,24433944.0,1.226258e+11,72648.0,3.645961e+08,1.829786e+12,216.0,1084032.0,5.440395e+09,2.730353e+13


In [50]:
feature_select = SequentialFeatureSelector(linear_model.LinearRegression(),
                                           scoring='neg_mean_squared_error',
                                           cv=[[training_indices, dev_indices]],
                                           n_features_to_select = 4)
feature_select.fit(all_degree_3_combinations, vehicle_data["price"])

SequentialFeatureSelector(cv=[[array([244222, 141804, 352601, ..., 353245, 355739, 129727]),
                               array([341562, 152239, 403459, ...,  62262, 143197, 280055])]],
                          estimator=LinearRegression(), n_features_to_select=4,
                          scoring='neg_mean_squared_error')

In [51]:
feature_select = SequentialFeatureSelector(estimator = linear_model.LinearRegression(),
                                           scoring='neg_mean_squared_error',
                                           cv=[[training_indices, dev_indices]],
                                           n_features_to_select = 4)

best_four = pd.DataFrame(feature_select.fit_transform(all_degree_3_combinations, vehicle_data["price"]),
                         columns = feature_select.get_feature_names_out())

In [52]:
best_four

,year,year^2,year^3,cylinders^3
0,2014.0,4056196.0,8.169179e+09,512.0
1,2010.0,4040100.0,8.120601e+09,512.0
2,2020.0,4080400.0,8.242408e+09,512.0
3,2017.0,4068289.0,8.205739e+09,512.0
4,2013.0,4052169.0,8.157016e+09,216.0
...,...,...,...,...
405657,2019.0,4076361.0,8.230173e+09,216.0
405658,2020.0,4080400.0,8.242408e+09,0.0
405659,2020.0,4080400.0,8.242408e+09,0.0
405660,2018.0,4072324.0,8.217950e+09,216.0


In [53]:
best_four_sfs_model = linear_model.LinearRegression()
best_four_sfs_model.fit(best_four.iloc[training_indices], vehicle_data.iloc[training_indices]["price"])

LinearRegression()

In [54]:
mean_squared_error(best_four_sfs_model.predict(best_four.iloc[training_indices]),
                                               vehicle_data.iloc[training_indices]["price"])

237606144.01810056

In [55]:
#8) Model Ridge with scaler 
ss = StandardScaler()
rescaled_df = pd.DataFrame(ss.fit_transform(all_degree_3_combinations),
                           columns = ss.get_feature_names_out())

In [56]:
rescaled_df

,year,cylinders,odometer,year^2,year cylinders,year odometer,cylinders^2,cylinders odometer,odometer^2,year^3,year^2 cylinders,year^2 odometer,year cylinders^2,year cylinders odometer,year odometer^2,cylinders^3,cylinders^2 odometer,cylinders odometer^2,odometer^3
0,0.280030,1.416462,-0.194416,0.280204,1.421687,-0.195473,1.737200,0.087500,-0.025665,0.280348,1.426777,-0.196488,1.743612,0.089549,-0.025733,1.873246,0.162121,-0.017322,-0.019144
1,-0.166716,1.416462,-0.127851,-0.170232,1.416684,-0.128789,1.737200,0.183162,-0.024706,-0.173745,1.416760,-0.129710,1.738300,0.185248,-0.024766,1.873246,0.265227,-0.015930,-0.019134
2,0.950149,1.416462,-0.388334,0.957537,1.429191,-0.391536,1.737200,-0.191183,-0.027333,0.964878,1.441841,-0.394664,1.751579,-0.191820,-0.027424,1.873246,-0.138246,-0.019742,-0.019154
3,0.615089,1.416462,-0.278456,0.618619,1.425439,-0.280258,1.737200,-0.033275,-0.026594,0.622104,1.434303,-0.281998,1.747595,-0.032124,-0.026674,1.873246,0.031949,-0.018670,-0.019151
4,0.168344,0.788144,0.156154,0.167511,0.791022,0.159176,0.570146,0.361251,-0.018391,0.166656,0.793812,0.162188,0.572786,0.366000,-0.018360,0.336189,0.271201,-0.010086,-0.019035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405657,0.838462,0.788144,-0.322970,0.844508,0.796650,-0.325235,0.570146,-0.155168,-0.026958,0.850507,0.805104,-0.327431,0.577268,-0.155383,-0.027043,0.336189,-0.146250,-0.019409,-0.019153
405658,0.950149,-1.096809,-0.424008,0.957537,-1.097221,-0.427766,-0.930350,-0.328932,-0.027457,0.964878,-1.097567,-0.431447,-0.930854,-0.331517,-0.027551,-0.785448,-0.286713,-0.020040,-0.019155
405659,0.950149,-1.096809,-0.463304,0.957537,-1.097221,-0.467673,-0.930350,-0.328932,-0.027528,0.964878,-1.097567,-0.471965,-0.930854,-0.331517,-0.027623,-0.785448,-0.286713,-0.020040,-0.019155
405660,0.726776,0.788144,-0.333545,0.731536,0.795712,-0.336046,0.570146,-0.166567,-0.027032,0.736249,0.803219,-0.338478,0.576521,-0.167018,-0.027118,0.336189,-0.155464,-0.019489,-0.019153


In [57]:
scaled_ridge_model = Pipeline([
    ('vincent_transform', PolynomialFeatures(degree = 3, include_bias = False)),
    ('scale', StandardScaler()),
    ('vincent_regression', Ridge())
])

parameters_to_try = {'vincent_regression__alpha': 10**np.linspace(-5, 4, 100)}

from sklearn.model_selection import GridSearchCV
model_finder = GridSearchCV(estimator = scaled_ridge_model,
                               param_grid = parameters_to_try,
                               scoring = "neg_mean_squared_error",
                               cv=[[training_indices, dev_indices]])

In [58]:
model_finder.fit(vehicle_data[numeric_features], vehicle_data["price"])

GridSearchCV(cv=[[array([244222, 141804, 352601, ..., 353245, 355739, 129727]),
                  array([341562, 152239, 403459, ...,  62262, 143197, 280055])]],
             estimator=Pipeline(steps=[('vincent_transform',
                                        PolynomialFeatures(degree=3,
                                                           include_bias=False)),
                                       ('scale', StandardScaler()),
                                       ('vincent_regression', Ridge())]),
             param_grid={'vincent_regression__alpha': array([1.00000000e-05, 1.2...
       8.11130831e+01, 1.00000000e+02, 1.23284674e+02, 1.51991108e+02,
       1.87381742e+02, 2.31012970e+02, 2.84803587e+02, 3.51119173e+02,
       4.32876128e+02, 5.33669923e+02, 6.57933225e+02, 8.11130831e+02,
       1.00000000e+03, 1.23284674e+03, 1.51991108e+03, 1.87381742e+03,
       2.31012970e+03, 2.84803587e+03, 3.51119173e+03, 4.32876128e+03,
       5.33669923e+03, 6.57933225e+03, 8.11130831e+03, 1.00000000e+04])},
             scoring='neg_mean_squared_error')

In [59]:
best_model = model_finder.best_estimator_
best_model

Pipeline(steps=[('vincent_transform',
                 PolynomialFeatures(degree=3, include_bias=False)),
                ('scale', StandardScaler()),
                ('vincent_regression', Ridge(alpha=1e-05))])

In [60]:
print(f"Dev MSE for our best Ridge Regression model: {-model_finder.best_score_}") #e4

Dev MSE for our best Ridge Regression model: 234667559.91138524


In [61]:
mean_squared_error(best_model.predict(training_vehicle_data[numeric_features]), training_vehicle_data["price"])

224538569.9544591

In [62]:
best_model.named_steps["vincent_regression"].coef_

array([ 4.28373581e+07,  1.46932283e+06, -8.90260137e+05, -8.67557726e+07,
       -2.47255742e+06,  2.00644071e+06, -6.63600151e+05,  1.45041480e+04,
       -1.94455938e+05,  4.39241588e+07,  9.81000227e+05, -1.11949437e+06,
        7.06299914e+05, -1.53137172e+04,  2.06870013e+05, -1.86745215e+04,
       -1.85137693e+03,  2.44512061e+03, -1.01865469e+04])

In [63]:
pd.DataFrame([best_model.named_steps["vincent_regression"].coef_],
             columns = best_model.named_steps["vincent_transform"].get_feature_names_out())

,year,cylinders,odometer,year^2,year cylinders,year odometer,cylinders^2,cylinders odometer,odometer^2,year^3,year^2 cylinders,year^2 odometer,year cylinders^2,year cylinders odometer,year odometer^2,cylinders^3,cylinders^2 odometer,cylinders odometer^2,odometer^3
0,4.283736e+07,1.469323e+06,-890260.136703,-8.675577e+07,-2.472557e+06,2.006441e+06,-663600.151003,14504.148047,-194455.937999,4.392416e+07,981000.226723,-1.119494e+06,706299.913789,-15313.717238,206870.013018,-18674.521455,-1851.376934,2445.120611,-10186.546867


In [ ]:
# 9) Model Lasso with scaler
scaled_lasso_model = Pipeline([
    ('vincent_transform', PolynomialFeatures(degree = 3, include_bias = False)),
    ('scale', StandardScaler()),
    ('vincent_lasso', Lasso())
])

parameters_to_try = {'vincent_lasso__alpha': 10**np.linspace(-4, 4, 100)}

model_finder = GridSearchCV(estimator = scaled_lasso_model,
                               param_grid = parameters_to_try,
                               scoring = "neg_mean_squared_error",
                               cv=[[training_indices, dev_indices]])
model_finder.fit(vehicle_data[numeric_features], vehicle_data["price"])

C:\Users\vince\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.017e+13, tolerance: 1.016e+10

C:\Users\vince\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.017e+13, tolerance: 1.016e+10

C:\Users\vince\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.017e+13, tolerance: 1.016e+10

C:\Users\vince\anaconda3\Lib\site-packages\sklearn\linear_model\_co

In [ ]:
model_finder.best_estimator_.named_steps['vincent_lasso'].coef_

In [ ]:
best_model = model_finder.best_estimator_

In [ ]:
lasso_weights = pd.DataFrame([best_model.named_steps["vincent_lasso"].coef_],
             columns = best_model.named_steps["vincent_transform"].get_feature_names_out())
lasso_weights

In [80]:
mean_squared_error(best_model.predict(training_vehicle_data[numeric_features]), training_vehicle_data["price"])

240746675.63264707

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [81]:
# In the previous step we used 9 models :
# 1) Model with simple linear regression with column year with fit_intercept = False
# mean squared error was : 309 197 835
# 2) Model with simple linear regression with column year with fit_intercept = True
# mean squared error was : 291 390 766
# 3) Model with multiple Linear Regression
# mean squared error was : 290 725 820
# 4) Model with Linear regression using Non-numeric Features
# mean squared error was : 282 145 130
# 5) Model using an SKLearn Pipeline and polynomial features and only 1 attribute
# mean squared error was : 242 712 645
# 6) Model using an SKLearn Pipeline and polynomial features and many attributes
# mean squared error was : 244 253 826
# 7) Model using SequentialFeatureSelection
# mean squared error was : 234 757 536
# 8) Model Ridge with scaler 
# Training mean squared error was : 221 660 886
# 9) Model Lasso with scaler
# mean squared error was : 240 746 675

# so it seems the model 8) Ridge with scaler was the best as it gives the lowest mean squared error.
# we can provide some insight to the user based on this model.
# however the mean squared error seems high as it is in the hundred of millions, so it might need to be # revisited to see if we can have it lower
# also i could spend more time to analyse other attributes to see what else can drive the price of the car

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.

In [82]:
# according to our Model Ridge with scaler, we can predict the price of the car based on the attribute "year", "cylinders", "odometer".
# when analyzing the data we noticed that the average of price are :
#- higher in UT, cheaper in ME
#- higher for white car, cheaper for green car
#- higher for pickup, cheaper for mini-van
#- higher for full_size, cheaper for compact
#- higher for 4wd drive and cheaper for fwd drive
#- the transmission type doesn't seem to impact too much the price
#- higher for lien title_status and cheaper for parts only or salvage
#- higher for diesel fuel and cheaper for hybrid fuel
#- higher in general for more cylinders
#- higher when condition is new, cheaper when condition is fair or salvage
#- higher for ferrari and cheaper for toyota
